In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LinearRegression


In [2]:
# ============================================================
# 0️⃣ Path & Load
# ============================================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data" / "processed"

MONTHLY_PATH = DATA_DIR / "train_monthly_v2.csv"
print("📂 LOAD:", MONTHLY_PATH)

df = pd.read_csv(MONTHLY_PATH)

# 안전하게 정렬
df = df.sort_values(["item_id", "year", "month"]).reset_index(drop=True)


# ============================================================
# 1️⃣ Helper: Trend 계산 함수 (Linear Regression)
# ============================================================
def compute_trend(values: np.ndarray):
    """
    values: 1D sequence of monthly values (log or raw)
    return: slope of linear regression
    """
    if len(values) < 3:
        return np.nan
    
    X = np.arange(len(values)).reshape(-1, 1)
    y = values
    try:
        lr = LinearRegression()
        lr.fit(X, y)
        return lr.coef_[0]
    except:
        return np.nan


# ============================================================
# 2️⃣ Item-level rolling & diff 계산
# ============================================================
df["roll_mean_3"]  = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(3).mean())
df["roll_mean_6"]  = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(6).mean())
df["roll_mean_12"] = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(12).mean())

df["roll_std_3"]  = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(3).std())
df["roll_std_6"]  = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(6).std())
df["roll_std_12"] = df.groupby("item_id")["total_value"].transform(lambda x: x.rolling(12).std())

# Δvalue & Δweight
df["diff_value"]  = df.groupby("item_id")["total_value"].diff()
df["diff_weight"] = df.groupby("item_id")["total_weight"].diff()

# diff 방향 (증가/감소 패턴)
df["sign_value"]  = np.sign(df["diff_value"])
df["sign_weight"] = np.sign(df["diff_weight"])


# ============================================================
# 3️⃣ item-level trend (log_value 기준)
# ============================================================
trend_list = []

for item, sub in df.groupby("item_id"):
    values = sub["log_value"].values
    slope = compute_trend(values)
    trend_list.extend([slope] * len(sub))

df["trend_log_value"] = trend_list


# ============================================================
# 4️⃣ Monthly seasonality index (item-level)
# ============================================================
# month별 평균 / 전체 평균
month_mean = df.groupby("month")["total_value"].mean()
global_mean = df["total_value"].mean()

seasonality_index = month_mean / global_mean
seasonality_index = seasonality_index.to_dict()

df["seasonality_item"] = df["month"].map(seasonality_index)


# ============================================================
# 5️⃣ HS4-level seasonality (더 안정적인 패턴)
# ============================================================
hs4_season = (
    df.groupby(["hs4", "month"])["total_value"]
      .mean()
      .reset_index()
      .rename(columns={"total_value": "hs4_month_avg"})
)

# normalize by hs4 yearly mean
hs4_global = (
    df.groupby("hs4")["total_value"].mean()
      .rename("hs4_global_mean")
      .reset_index()
)

hs4_season = hs4_season.merge(hs4_global, on="hs4", how="left")
hs4_season["hs4_season_idx"] = hs4_season["hs4_month_avg"] / hs4_season["hs4_global_mean"]

df = df.merge(hs4_season[["hs4", "month", "hs4_season_idx"]],
              on=["hs4", "month"], how="left")


# ============================================================
# 6️⃣ rare item flag (EDA)
# ============================================================
item_total = df.groupby("item_id")["total_value"].sum()
rare_threshold = item_total.quantile(0.05)
rare_items = item_total[item_total < rare_threshold].index

df["rare_item_flag"] = df["item_id"].isin(rare_items).astype(int)


# ============================================================
# 7️⃣ 패널 보강 (optional)
# ============================================================
# (item_id, year, month) 누락된 경우 value=0 채워 넣는 버전도 만들기 옵션
# 일단 Step1 EDA는 원본 그대로 사용

# ============================================================
# 8️⃣ Save for Step1 EDA visualization
# ============================================================
SAVE_PATH = DATA_DIR / "train_monthly_step1prep.csv"
df.to_csv(SAVE_PATH, index=False)

print("💾 Step1 EDA 준비 파일 저장 완료:", SAVE_PATH)
print("🎉 rolling / diff / trend / seasonality 계산 완료!")


📂 LOAD: /data/ephemeral/home/data/processed/train_monthly_v2.csv
💾 Step1 EDA 준비 파일 저장 완료: /data/ephemeral/home/data/processed/train_monthly_step1prep.csv
🎉 rolling / diff / trend / seasonality 계산 완료!
